In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import time
import json
import csv

In [2]:
import pandas as pd
urls_df = pd.read_csv('./archive/UE-NY-rest-urls.csv')
urls_df.head()

,url,city
0,https://www.ubereats.com/new-york/food-deliver...,new-york
1,https://www.ubereats.com/new-york/food-deliver...,new-york
2,https://www.ubereats.com/new-york/food-deliver...,new-york
3,https://www.ubereats.com/new-york/food-deliver...,new-york
4,https://www.ubereats.com/new-york/food-deliver...,new-york


In [3]:
def get_rest_details(rest_url):
    start_time = time.time()
    rest_obj = {}
    try:
        req = Request(rest_url, headers={'User-Agent': 'Mozilla/5.0', 'Accept': 'text/html,application/xhtml+xml,application/xml'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, 'html.parser')
        rest_obj = json.loads(soup.find("script", type="application/ld+json").text)
#         print(rest_obj['name'])
    except Exception as e:
        print(f"\nerror: {e}\n url: {rest_url}")
    
    elapsed_time = time.time()-start_time
#     print(f"elapsed time: {elapsed_time}")
    return rest_obj

In [4]:
def scrap_all_rest(urls_df, f, error_rests):
    start_time = time.time()
    num_rests = urls_df.shape[0]
    writer = csv.writer(f, delimiter=',')
    for i in range(num_rests):
        curr_rest = urls_df.iloc[i]
        if i % 50 == 0:
                print(f"Progress: {(i*100)//num_rests}%, restaurants done: {i}, time elapsed: {time.time()-start_time}")
                
        try:
            obj = get_rest_details(curr_rest.url)
            writer.writerow([
                obj['name'].replace(';', ''),
                curr_rest.city,
                obj.get('servesCuisine'),
                obj.get('aggregateRating'),
                obj.get('priceRange'),
                obj.get('geo'),
                obj.get('telephone'),
                curr_rest.url,
                obj.get('address'),
                obj.get('openingHoursSpecification'),
            ])
        except Exception as e:
            print(f"\nerror: {e}\n url: {curr_rest.url}")
            error_rests.append(curr_rest.url)

    elapsed_time = time.time()-start_time
    print(f"total time: {elapsed_time}, error count: {len(error_rests)}")
    return error_rests

In [5]:
f = open('./UE-NY-rest-details.csv', 'a')
error_rests = []
scrap_all_rest(urls_df, f, error_rests)
# curr_rest = urls_df.iloc[0]
# curr_rest.url

Progress: 0%, restaurants done: 0, time elapsed: 0.0003151893615722656
Progress: 1%, restaurants done: 50, time elapsed: 120.46354985237122
Progress: 3%, restaurants done: 100, time elapsed: 230.5447268486023
Progress: 4%, restaurants done: 150, time elapsed: 341.4672374725342
Progress: 6%, restaurants done: 200, time elapsed: 465.8680408000946
Progress: 8%, restaurants done: 250, time elapsed: 570.3720235824585
Progress: 9%, restaurants done: 300, time elapsed: 674.2486836910248
Progress: 11%, restaurants done: 350, time elapsed: 783.4704251289368
Progress: 12%, restaurants done: 400, time elapsed: 890.1043407917023
Progress: 14%, restaurants done: 450, time elapsed: 1018.0634372234344
Progress: 16%, restaurants done: 500, time elapsed: 1124.1802880764008
Progress: 17%, restaurants done: 550, time elapsed: 1264.4255456924438
Progress: 19%, restaurants done: 600, time elapsed: 1380.0564126968384
Progress: 20%, restaurants done: 650, time elapsed: 1515.7142720222473
Progress: 22%, resta

KeyboardInterrupt: 

In [7]:
error_rests

['https://www.ubereats.com/new-york/food-delivery/the-snack-pusher-murray-hill/80WsNn47TVeQ-AQM_BLuBg',
 'https://www.ubereats.com/new-york/food-delivery/cafe-oliviero/jpppMJulSWiCJFn1_K4iMg',
 'https://www.ubereats.com/new-york/food-delivery/city-burger-co/XrWTliv2QM2GFefDHTdRkg',
 'https://www.ubereats.com/new-york/food-delivery/the-ice-cream-shop/2y6yFh_HS4q5isxaScOFFg']

In [21]:
url = urls_df.iloc[2613]['url']
obj = get_rest_details(url)

In [18]:
obj

{'@context': 'http://schema.org',
 '@id': 'https://www.ubereats.com/new-york/food-delivery/grandlo-cafe/YYHa-1X9SumDbHHS2bUQZQ',
 '@type': 'Restaurant',
 'address': {'@type': 'PostalAddress',
  'addressCountry': 'US',
  'addressLocality': 'New York',
  'addressRegion': 'NY',
  'postalCode': '10002',
  'streetAddress': '168 Broome St'},
 'aggregateRating': {},
 'geo': {'@type': 'GeoCoordinates',
  'latitude': 40.7168819,
  'longitude': -73.9857066},
 'image': ['https://d1ralsognjng37.cloudfront.net/441cca2f-daf1-4896-af8d-d69d0f126d45.jpeg',
  'https://d1ralsognjng37.cloudfront.net/8cde219d-81d4-4c05-9720-40f28c6d5e0c.jpeg',
  'https://d1ralsognjng37.cloudfront.net/6e748356-226a-4816-b82c-2df3724ee5cf.jpeg',
  'https://d1ralsognjng37.cloudfront.net/6f5b0624-f909-4717-bfdb-b50c32972d49.jpeg',
  'https://d1ralsognjng37.cloudfront.net/01498a40-70da-492f-be87-857681ee80a2.jpeg',
  'https://d1ralsognjng37.cloudfront.net/14d074ea-fabb-4ea8-a55e-cc8d34976fdc.jpeg'],
 'name': 'GrandLo Cafe',
 '